In [17]:
import asf_search as asf
import numpy as np
import geopandas as gpd
import pandas as pd
# import holoviews as hv
# from holoviews import opts
# hv.extension('bokeh')
import datetime
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure,show

In [40]:
def make_df(results):
    df = pd.DataFrame(
    {
        'fileID':[],
        'platform':[],
        'flightDirection':[],
        'centerLat':[],
        'centerLon':[],
        'processingLevel':[],
        'startTime':[]
    })
    name = []
    platformlist = []
    flightlist = []
    latlist = []
    lonlist = []
    processlist = []
    stime = []
    for i in range(len(results)):
        name.append(results[i].properties['fileID'])
        platformlist.append(results[i].properties['platform'])
        flightlist.append(results[i].properties['flightDirection'])
        latlist.append(float(results[i].properties['centerLat']))
        lonlist.append(float(results[i].properties['centerLon']))
        processlist.append(results[i].properties['processingLevel'])
        timestring = results[i].properties['startTime'][0:10] + ' ' + results[i].properties['startTime'][11:]
        stime.append(timestring)

    df.fileID = name
    df.platform = platformlist
    df.flightDirection = flightlist
    df.centerLat = latlist
    df.centerLon = lonlist
    df.processingLevel = processlist
    df.startTime = stime
    
    return df

In [42]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 100,
    'start': '2016-01-01T00:00:00Z',
    'end': '2016-12-31T23:59:59Z'
}
results = asf.search(**opts)

df_2016 = make_df(results)
gdf = gpd.GeoDataFrame(df_2016, geometry=gpd.points_from_xy(df_2016.centerLon, df_2016.centerLat))

In [25]:
gdf.head()

,fileID,platform,flightDirection,centerLat,centerLon,processingLevel,startTime
0,S1B_EW_GRDM_1SDH_20161231T235652_20161231T2357...,Sentinel-1B,DESCENDING,74.7488,106.1149,GRD_MD,2016-12-31 23:56:52.000000
1,S1B_EW_GRDM_1SDH_20161231T235652_20161231T2357...,Sentinel-1B,DESCENDING,74.7488,106.1149,METADATA_GRD_MD,2016-12-31 23:56:52.000000
2,S1B_EW_RAW__0SDH_20161231T235648_20161231T2357...,Sentinel-1B,DESCENDING,74.7983,106.3775,METADATA_RAW,2016-12-31 23:56:48.000000
3,S1B_EW_RAW__0SDH_20161231T235648_20161231T2357...,Sentinel-1B,DESCENDING,74.7983,106.3775,RAW,2016-12-31 23:56:48.000000
4,S1B_EW_RAW__0SDH_20161231T235548_20161231T2356...,Sentinel-1B,DESCENDING,78.0089,111.4157,METADATA_RAW,2016-12-31 23:55:48.000000


In [43]:
type(df_2016.centerLat.iloc[0])

numpy.float64

In [24]:
gdf.drop(['geometry'], axis=1,inplace=True)

In [26]:
gdf.to_csv('Sentinel1_2016.csv')

In [6]:
def get_dataset(src, name):
    df = src.copy()
    df['date'] = pd.to_datetime(df.startTime)
    # timedelta here instead of pd.DateOffset to avoid pandas bug < 0.18 (Pandas issue #11925)
#     df['left'] = df.date - datetime.timedelta(days=0.5)
#     df['right'] = df.date + datetime.timedelta(days=0.5)
    df = df.set_index(['date'])
    df.sort_index(inplace=True)

    return ColumnDataSource(data=df)

In [44]:
def make_plot(source, title):
#     plot = figure(x_axis_type="datetime", width=800, tools="", toolbar_location=None)
    plot = figure()
    plot.title.text = title
    

#     plot.circle('date','centerLat',source=source)

    plot.circle(df_2016.startTime,df_2016.centerLat)

    # fixed attributes
    plot.xaxis.axis_label = None
    plot.yaxis.axis_label = "Latitude"
    plot.axis.axis_label_text_font_style = "bold"
    plot.x_range = DataRange1d(range_padding=0.0)
    plot.grid.grid_line_alpha = 0.3

    return plot

In [7]:
source = get_dataset(df_2016,df_2016.fileID)

In [45]:
plot = make_plot(source, "Ground-swath project")

In [46]:
curdoc().add_root(row(plot))
curdoc().title = "Ground-swath project"